In [1]:
# Imoort Libraries
import os
import pandas as pd
import seaborn as sns
import env
import wrangle
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Mask for American Airlines

In [2]:
# Using the 'wrangle' module, this line pulls airline data for the airline code 'AA' (American Airlines) and stores it in the 'flights' variable.
flights = wrangle.pull_airline_data('AA')

In [3]:
# This line of code counts the number of occurrences for each unique carrier (OP_CARRIER) in the 'flights' dataset. 
# It will return a Series with the carriers as the index and the counts as the values, sorted in descending order by default.

flights.OP_CARRIER.value_counts()

AA    1050420
Name: OP_CARRIER, dtype: int64

In [31]:
# Display the first few rows of the 'flights' DataFrame transposed, 
# allowing for an easier view of columns, especially if there are many columns.

flights.head(100).T

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
FL_DATE,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,...,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00
OP_CARRIER,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA,...,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA
OP_CARRIER_FL_NUM,2,16,75,76,85,117,118,127,162,178,...,1432,1433,1441,1467,1475,1494,1517,1530,1532,1554
ORIGIN,LAX,SFO,IAD,LAX,JFK,JFK,LAX,JFK,LAX,SFO,...,DFW,DFW,ORD,EWR,DFW,SEA,DFW,MIA,DFW,DFW
DEST,JFK,JFK,LAX,IAD,SFO,LAX,JFK,SAN,MIA,JFK,...,EWR,SNA,SEA,MIA,LAS,DFW,ABQ,LGA,AUS,STL
CARRIER_DELAY,35.0,83.0,35.0,15.0,26.0,100.0,12.0,6.0,22.0,15.0,...,12.0,0.0,0.0,134.0,33.0,165.0,20.0,23.0,0.0,20.0
WEATHER_DELAY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NAS_DELAY,0.0,0.0,0.0,0.0,0.0,0.0,8.0,24.0,2.0,0.0,...,0.0,0.0,27.0,34.0,0.0,0.0,2.0,0.0,15.0,0.0
SECURITY_DELAY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LATE_AIRCRAFT_DELAY,0.0,0.0,7.0,0.0,16.0,0.0,0.0,0.0,76.0,37.0,...,6.0,9.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0


In [5]:
# Using the 'wrangle' module, clean the 'flights' dataframe to prepare it for computing the average daily delay.
df = wrangle.clean_flight_data_for_average_daily_delay(flights)


In [6]:
# List of columns representing different types of flight delays to be summed up for analysis.
col_to_sum = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']


In [9]:
# Display the first five rows of the 'flights' DataFrame to get an initial overview of its structure and data.
flights.head().T



,0,1,2,3,4
FL_DATE,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00
OP_CARRIER,AA,AA,AA,AA,AA
OP_CARRIER_FL_NUM,2,16,75,76,85
ORIGIN,LAX,SFO,IAD,LAX,JFK
DEST,JFK,JFK,LAX,IAD,SFO
CARRIER_DELAY,35.0,83.0,35.0,15.0,26.0
WEATHER_DELAY,0.0,0.0,0.0,0.0,0.0
NAS_DELAY,0.0,0.0,0.0,0.0,0.0
SECURITY_DELAY,0.0,0.0,0.0,0.0,0.0
LATE_AIRCRAFT_DELAY,0.0,0.0,7.0,0.0,16.0


In [8]:
# This section deals with data or operations related to flights, which can include details like departure, arrival, flight duration, airline information, and other related attributes.
flights


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,row_sums
0,2009-01-01,AA,2,LAX,JFK,35.0,0.0,0.0,0.0,0.0,35.0
1,2009-01-01,AA,16,SFO,JFK,83.0,0.0,0.0,0.0,0.0,83.0
2,2009-01-01,AA,75,IAD,LAX,35.0,0.0,0.0,0.0,7.0,42.0
3,2009-01-01,AA,76,LAX,IAD,15.0,0.0,0.0,0.0,0.0,15.0
4,2009-01-01,AA,85,JFK,SFO,26.0,0.0,0.0,0.0,16.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...
1050415,2019-12-31,AA,1357,MIA,JFK,38.0,0.0,0.0,0.0,41.0,79.0
1050416,2019-12-31,AA,1358,DFW,SAT,51.0,0.0,0.0,0.0,0.0,51.0
1050417,2019-12-31,AA,1368,MIA,LAS,0.0,54.0,0.0,0.0,0.0,54.0
1050418,2019-12-31,AA,1376,DFW,EYW,0.0,0.0,0.0,0.0,24.0,24.0


In [32]:
# This code calculates and prints the percentage of on-time flights based on the absence of carrier delays. 
# If the 'CARRIER_DELAY' column has a missing (NaN) value, it's assumed that the flight was on time. 
# The percentage is computed by dividing the number of NaN values by the total number of flights, 
# then multiplying by 100. The result is formatted to two decimal places.
print(f'Airline On Time Percentage: {flights.CARRIER_DELAY.isna().sum()/len(flights) *100: .2f}%')


Airline On Time Percentage:  0.00%


In [10]:
# Display the first 5 rows of the dataframe 'df' to get an overview of its structure and data.
df.head()


,average_delay
FL_DATE,
2009-01-01,58.260274
2009-01-02,65.522222
2009-01-03,66.687151
2009-01-04,50.191589
2009-01-05,69.574074


In [11]:
# Retrieve the number of rows and columns in the 'flights' DataFrame.
flights.shape


(1050420, 11)

In [13]:
# Generate a statistical summary of the 'flights' dataframe, 
# providing an overview of the distribution for each numerical column.

flights.describe().T


,count,mean,std,min,25%,50%,75%,max
OP_CARRIER_FL_NUM,1050420.0,1366.505955,770.537902,1.0,663.0,1395.0,1990.0,2949.0
CARRIER_DELAY,1050420.0,20.382211,46.774431,0.0,0.0,4.0,24.0,2142.0
WEATHER_DELAY,1050420.0,3.469107,20.483603,0.0,0.0,0.0,0.0,1587.0
NAS_DELAY,1050420.0,13.798474,25.466529,0.0,0.0,3.0,18.0,1138.0
SECURITY_DELAY,1050420.0,0.087493,2.628331,0.0,0.0,0.0,0.0,987.0
LATE_AIRCRAFT_DELAY,1050420.0,19.962781,42.201950,0.0,0.0,0.0,24.0,1514.0
row_sums,1050420.0,57.700065,64.279035,15.0,23.0,38.0,69.0,2142.0


In [17]:
# Calculate the total delay for each row by summing the delays across the specified columns.

row_sums = flights[['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].sum(axis=1)
row_sums

0          35.0
1          83.0
2          42.0
3          15.0
4          42.0
           ... 
1050415    79.0
1050416    51.0
1050417    54.0
1050418    24.0
1050419    35.0
Length: 1050420, dtype: float64

In [18]:
# List of top 15 busiest airport hubs in the U.S. by airport code

top_15_hubs = ['ATL',  # Hartsfield-Jackson Atlanta International Airport
               'DFW',  # Dallas/Fort Worth International Airport
               'DEN',  # Denver International Airport
               'ORD',  # O'Hare International Airport, Chicago
               'LAX',  # Los Angeles International Airport
               'JFK',  # John F. Kennedy International Airport, New York
               'IAH',  # George Bush Intercontinental Airport, Houston
               'PHX',  # Phoenix Sky Harbor International Airport
               'EWR',  # Newark Liberty International Airport
               'SFO',  # San Francisco International Airport
               'SEA',  # Seattle-Tacoma International Airport
               'IAD',  # Washington Dulles International Airport
               'PHL',  # Philadelphia International Airport
               'CLT',  # Charlotte Douglas International Airport
               'MIA']  # Miami International Airport


In [19]:
# Extracting the frequency counts of each unique airport in the 'ORIGIN' column of the 'flights' dataframe.
# The result is stored in 'list_of_airports' which is then converted into a DataFrame.
# Finally, we're finding the number of unique counts (i.e., the number of times each airport frequency occurs) using 'nunique()'.

list_of_airports = flights['ORIGIN'].value_counts()
list_of_airports = pd.DataFrame(list_of_airports)
list_of_airports.nunique()


ORIGIN    15
dtype: int64

In [20]:
# Display the first 15 rows of the 'list_of_airports' dataframe to get a quick overview of its data structure and contents.
list_of_airports.head(15)


,ORIGIN
DFW,383029
ORD,148995
MIA,126564
CLT,89035
LAX,69633
PHX,49937
PHL,48832
JFK,35101
SFO,28341
DEN,15886


### Investigate the relationship between these columns and determine methods for merging them.

### Evaluate the columns using different criteria and metrics.


In [21]:
#Fills in nulls as zero as null means no delay
flights.fillna(0, inplace=True)
#Makes FL_DATE column a datetime datatype
flights.FL_DATE = flights.FL_DATE.astype('datetime64')
#Makes FL_DATE the index
# flights = flights.set_index('FL_DATE')

In [22]:
flights_cat = flights
flights_cat['carrier'] = flights_cat.CARRIER_DELAY > 0

In [23]:
flights_cat['weather'] = flights_cat.WEATHER_DELAY > 0

In [24]:
flights_cat['nas'] = flights_cat.NAS_DELAY > 0

In [25]:
flights_cat['security'] = flights_cat.SECURITY_DELAY > 0

In [26]:
flights_cat['late_aircraft'] = flights_cat.LATE_AIRCRAFT_DELAY > 0

In [27]:
flights_cat.drop(columns=['OP_CARRIER', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'])

,FL_DATE,OP_CARRIER_FL_NUM,ORIGIN,DEST,row_sums,carrier,weather,nas,security,late_aircraft
0,2009-01-01,2,LAX,JFK,35.0,True,False,False,False,False
1,2009-01-01,16,SFO,JFK,83.0,True,False,False,False,False
2,2009-01-01,75,IAD,LAX,42.0,True,False,False,False,True
3,2009-01-01,76,LAX,IAD,15.0,True,False,False,False,False
4,2009-01-01,85,JFK,SFO,42.0,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...
1050415,2019-12-31,1357,MIA,JFK,79.0,True,False,False,False,True
1050416,2019-12-31,1358,DFW,SAT,51.0,True,False,False,False,False
1050417,2019-12-31,1368,MIA,LAS,54.0,False,True,False,False,False
1050418,2019-12-31,1376,DFW,EYW,24.0,False,False,False,False,True


In [28]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050420 entries, 0 to 1050419
Data columns (total 16 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   FL_DATE              1050420 non-null  datetime64[ns]
 1   OP_CARRIER           1050420 non-null  object        
 2   OP_CARRIER_FL_NUM    1050420 non-null  int64         
 3   ORIGIN               1050420 non-null  object        
 4   DEST                 1050420 non-null  object        
 5   CARRIER_DELAY        1050420 non-null  float64       
 6   WEATHER_DELAY        1050420 non-null  float64       
 7   NAS_DELAY            1050420 non-null  float64       
 8   SECURITY_DELAY       1050420 non-null  float64       
 9   LATE_AIRCRAFT_DELAY  1050420 non-null  float64       
 10  row_sums             1050420 non-null  float64       
 11  carrier              1050420 non-null  bool          
 12  weather              1050420 non-null  bool          
 1

### At first glance it seems weird that weather does not have as many rows as the rest.  As a pilot I thought that weather dominates the delays so I wondered why this is the case.

### It seems that the NAS data includes non extreme weater as does carrier and late aircraft delay.

In [ ]:
dep = pd.crosstab(flights_cat.FL_DATE, [flights_cat.carrier, flights_cat.weather, flights_cat.nas, flights_cat.security, flights_cat.late_aircraft])

### Chi squared
* Null Hypothesis:  These delay types are independent of each other
* Alternative Hypothesis:  These types are dependent on each other
* Alpha < 0.05

In [ ]:
dep

In [ ]:
chi2, p, degf, expected = stats.chi2_contingency(dep)

In [ ]:
chi2, p

#### There is sufficient evidence to reject the null.  They are dependent on each other.

#### This suggests that there may bee some sort of overlap.  A quick look at the NAS delays shows that minor weather phenomena show up in the NAS as opposed to weather.  Also a weather delay on one leg of an airplanes trip seems to turn into a carrier delay on the next.

### A look at histograms

In [ ]:
plt.hist(flights.CARRIER_DELAY, bins=100)

In [ ]:
plt.hist(flights.WEATHER_DELAY, bins=100)

In [ ]:
plt.hist(flights.NAS_DELAY, bins=100)

In [ ]:
plt.hist(flights.SECURITY_DELAY, bins=100)

In [ ]:
plt.hist(flights.LATE_AIRCRAFT_DELAY, bins=100)

#### Skewed makes a whole lot of sense.  Nothing unexpected here.

### THERE IS sufficient evidence that I can combine these columns in feature engineering.

### Resample my data by daily mean delays

In [ ]:
flights_daily_mean = wrangle.clean_flight_data_for_average_daily_delay(flights)

In [ ]:
flights_daily_mean.sort_index()

In [ ]:
flights_daily_mean.isna().sum()

### Find the best interval to define average delay by to optimize reducing noise while keeping good correlation results

In [ ]:
def plot_average_by_interval():
    
    resample_dict = {'D':'Daily', 'W':'Weekly', '2W':'FORTNIGHTLY', 'M':'Monthly', '3M':'Quarterly', '6M':'Semi-Annually', 'Y':'Yearly'}

    for _ in resample_dict:
        plt.figure(figsize=(10,6))

        flights_daily_mean.resample(_).mean().average_delay.plot(label='daily')
        # flights_daily_mean.resample('W').mean().average_delay.plot(label='weekly')
        # flights_daily_mean.resample('M').mean().average_delay.plot(label='monthly')
        # flights_daily_mean.resample('Y').mean().average_delay.plot(label='yearly')


        plt.title(f'{resample_dict[_][0:]} Average Delay')
        plt.legend()
        plt.show()

        pd.plotting.autocorrelation_plot(flights_daily_mean.average_delay.resample(_).mean())
        plt.show()

plot_average_by_interval()

### Plot the best lag plot based on the airline.  The best will always be 365

In [ ]:
def plot_best_lag_plot():

    #Plots the best correlated lag
    pd.plotting.lag_plot(flights_daily_mean.resample('W').mean(), lag=365)
    plt.title('lag plot');

plot_best_lag_plot()

### Split the data by year

In [ ]:
train = flights_daily_mean.iloc[:2016] 
test = flights_daily_mean.iloc[2016:] 
train.loc['2009-09-30':]

In [ ]:
test

### Plot the seasonal decompositions for the various time periods

In [ ]:
y = train.average_delay.resample('d').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('w').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
def seasonal_decomposition_plot(duration):
    y = train.average_delay.resample(duration).mean()

    result = sm.tsa.seasonal_decompose(y)

    decomposition = pd.DataFrame({
        'y': result.observed,
        'trend': result.trend,
        'seasonal': result.seasonal,
        'resid': result.resid
    })

    decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
    decomposition[['trend_centered', 'seasonal', 'resid']].plot();
    
seasonal_decomposition_plot('2w')

In [ ]:
y = train.average_delay.resample('m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('3m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('6m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('y').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

### Once again the best period to resample by seems to be 2 weeks or a fortnight